In [30]:
import json
import time
from datetime import date
import pprint
from os import listdir
from feature_extract import feature_extract

HOUR_INDEX = 3
MINUTE_INDEX = 4
SECOND_INDEX = 5
    
files = listdir('../tweet_data')
print(files)

from os.path import splitext

import re

for fname in files:
    m = re.search('#.+', splitext(files[0])[0])
    if m:
        hashtag = m.group()
        print("Processing '%s'" % hashtag)
    outfile = splitext(fname)[0] + '.csv'
    print("Output file name '%s'" % outfile)
    
    f_handle = open('../tweet_data/'+fname, encoding="utf8")
    feature_extract(f_handle, outfile)
    f_handle.close()
    break;

['tweets_#gohawks.txt', 'tweets_#gopatriots.txt', 'tweets_#nfl.txt', 'tweets_#patriots.txt', 'tweets_#sb49.txt', 'tweets_#superbowl.txt']
Processing '#gohawks'
Output file name 'tweets_#gohawks.csv'
Starting from time 2014-12-28 14:00:00
Finished processing 1 tweets from hour 14, next date time 2014-12-29 06:40:08
Finished processing 1 tweets from hour 6, next date time 2014-12-29 07:27:13
Finished processing 1 tweets from hour 7, next date time 2014-12-29 11:27:20
Finished processing 1 tweets from hour 11, next date time 2014-12-29 12:10:38
Finished processing 1 tweets from hour 12, next date time 2014-12-29 16:39:54
Finished processing 1 tweets from hour 16, next date time 2014-12-30 10:03:35
Finished processing 1 tweets from hour 10, next date time 2014-12-30 16:23:47
Finished processing 2 tweets from hour 16, next date time 2014-12-31 08:20:51
Finished processing 1 tweets from hour 8, next date time 2014-12-31 11:06:19
Finished processing 1 tweets from hour 11, next date time 2014-

In [ ]:
import json
import time
from datetime import date

HOUR_INDEX = 3
MINUTE_INDEX = 4
SECOND_INDEX = 5

def feature_extract(f, o_file):
    import json
    import time
    from datetime import date

    line = f.readline()
    tweet = json.loads(line)

    n_tweets = 0 # number of tweets per hour
    n_followers = {} # uid -> (nfollower, ntweets) per hour
    n_retweets = 0 # number of retweets per hour
    num_window = 0 # total number of hours

    user_follower = {}

    # self defined features (for part 2)
    # n_users:-  number of users tweeted per hour
    # n_users_3:- number of users who tweeted 3 times or more per hour
    n_len_100 = 0 # number of tweets with more than 100 characters per hour

    # outputs
    import numpy as np

    """
    output: n_tweets  n_retweets  sum_followers  max_followers  hour \
            avg_tweet_per_user  n_users  n_users_3  n_len_100
    """
    output = np.empty([0, 9])

    # get start hour from first tweet
    start_time = list(time.localtime(tweet['firstpost_date']))
    start_time[MINUTE_INDEX] = 0; start_time[SECOND_INDEX] = 0
    nth_hour = start_time[HOUR_INDEX]
    start_time = time.mktime(start_time)
    start_date = date.fromtimestamp(start_time)
    print "Starting from time %s" % time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(start_time))

    time_format = "%Y-%m-%d %H:%M:%S"

    while len(line) != 0:
        tweet = json.loads(line)
        line = f.readline()
        tweet_time = tweet['firstpost_date']
        tweet_hour = time.localtime(tweet_time).tm_hour
        if tweet_hour != nth_hour or start_date != date.fromtimestamp(float(tweet_time)):
            print "Finished processing %d tweets from hour %d, next date time %s" %\
                  (n_tweets, nth_hour, time.strftime(time_format, time.localtime(tweet_time)))
            # update output
            tweets_per_user = np.array(n_followers.values())[:, 1]
            new_hour = np.array([[n_tweets, n_retweets, sum(n_followers), max(n_followers), nth_hour, \
                                  np.mean(tweets_per_user), len(n_followers), sum(tweets_per_user >= 3), \
                                  n_len_100]])
            output = np.append(output, new_hour, axis=0)
            # clear & update hourly variables
            num_window += 1
            n_tweets = 0 # number of tweets per hour
            n_followers = {} # number of followers of users posting the tweets per hour
            n_retweets = 0 # number of retweets per hour
            nth_hour = tweet_hour  # hour of the day
            n_len_100 = 0

        # extract features
        n_tweets += 1
        n_retweets += tweet['tweet']['retweet_count']
        if tweet['tweet']['user']['id'] in n_followers:
            # increment user tweet count
            tweet_count = n_followers[tweet['tweet']['user']['id']][1] + 1
            n_followers[tweet['tweet']['user']['id']] = (tweet['tweet']['user']['followers_count'], tweet_count)
        else:
            n_followers[tweet['tweet']['user']['id']] = (tweet['tweet']['user']['followers_count'], 1)
        if (len(tweet['tweet']['text']) >= 100): n_len_100 += 1
        user_follower[tweet['tweet']['user']['id']] = tweet['tweet']['user']['followers_count']
        start_date = date.fromtimestamp(float(tweet_time))

    print "Outputting to '%s' ..." % o_file
    np.savetxt('data/' + o_file, output, delimiter=',')

    print "--------------------"
    print "Total number of tweets %d" % int(np.sum(output[:, 0]))
    print "Average number of tweets per hour %f" % np.mean(output[:, 0])
    print "Average number of retweets %f" % float(np.sum(output[:, 1]) / np.sum(output[:, 0]))
    print "Average number of followers of (%d) users %f" %\
          (len(user_follower), float(sum(user_follower.values()) / float(len(user_follower))))

In [8]:
import matplotlib.pyplot as plt
import numpy as np

history = np.loadtxt('data/#SuperBowlhistogram')
hours = np.loadtxt('data/#SuperBowlhistogramlabel')

plt.cla()
#plt.yscale('log')
plt.plot(hours, history)
plt.title('#SuperBowl Hourly Tweet Count')
plt.xlabel('Hour')
plt.ylabel('Number of Tweets')

plt.xticks(range(len(hours))[::100], hours[::100])
plt.savefig('graphs/#SuperBowl_tweet_count_histogram')
plt.show()

In [7]:
import matplotlib.pyplot as plt
import numpy as np

history = np.loadtxt('data/#NFLhistogram')
hours = np.loadtxt('data/#NFLhistogramlabel')

plt.cla()
#plt.yscale('log')
plt.plot(hours, history)
plt.title('#NFL Hourly Tweet Count')
plt.xlabel('Hour')
plt.ylabel('Number of Tweets')

plt.xticks(range(len(hours))[::100], hours[::100])
plt.savefig('graphs/#NFL_tweet_count_histogram')
plt.show()

In [9]:
### Problem 2 and 3
from os import listdir
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression as LR
from sklearn.linear_model import LassoLars as lasso
from sklearn.linear_model import LogisticRegression as LogR
from sklearn.feature_selection import f_regression as freg
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# directory name
path = 'data/'

# hashtag name
hashtag = []

# csv file name
csv = []

files = listdir(path)

# append csv names into list
for f in files:
    if f.endswith(".csv"):
        hashtag.append(f[7:-4])
        csv.append(path + f)

# start the training per hashtag file

col_names = ["n_tweets", "n_retweets", "sum_followers", "max_followers", "hour", "avg_tweet_per_user", "n_users", "n_users_3", "n_len_100"]
for c in csv:
    print(c + "\n")
    csv_df = pd.read_csv(c, header=None, names=col_names)
    # aggregate by hour of day
    csv_df.sort_values(by='hour')
    
    LR2score = []
    LR2fval = np.array([0,0,0,0])
    LR2pval = np.array([0,0,0,0])
    
    LR3score = []
    LR3fval = np.array([0,0,0,0,0,0,0,0])
    LR3pval = np.array([0,0,0,0,0,0,0,0])
    
    LogR2score = []
    LogR2fval = np.array([0,0,0,0])
    LogR2pval = np.array([0,0,0,0])
    
    LogR3score = []
    LogR3fval = np.array([0,0,0,0,0,0,0,0])
    LogR3pval = np.array([0,0,0,0,0,0,0,0])
    
    SVR2score = []
    SVR2fval = np.array([0,0,0,0])
    SVR2pval = np.array([0,0,0,0])
    
    SVR3score = []
    SVR3fval = np.array([0,0,0,0,0,0,0,0])
    SVR3pval = np.array([0,0,0,0,0,0,0,0])
    
    Lasso2score = []
    Lasso2fval = np.array([0,0,0,0])
    Lasso2pval = np.array([0,0,0,0])
    
    Lasso3score = []
    Lasso3fval = np.array([0,0,0,0,0,0,0,0])
    Lasso3pval = np.array([0,0,0,0,0,0,0,0])
    
    for hr in range(0,23):
        #train data setup
        data_prev = csv_df[csv_df['hour'] == hr].copy()
        #features
        data2X_train = data_prev[['n_retweets','sum_followers', 'max_followers', 'hour']].copy()
        data3X_train = data_prev[['n_retweets','sum_followers', 'max_followers', 'hour', 'avg_tweet_per_user', 'n_users', 'n_users_3', 'n_len_100']].copy()
        #targets
        data2Y_train = data_prev[['n_tweets']].copy()
        data3Y_train = data_prev[['n_tweets']].copy()
        
        #test data setup
        data_next = csv_df[csv_df['hour'] == hr+1].copy()
        #features
        data2X_test = data_next[['n_retweets','sum_followers', 'max_followers', 'hour']].copy()
        data3X_test = data_next[['n_retweets','sum_followers', 'max_followers', 'hour', 'avg_tweet_per_user', 'n_users', 'n_users_3', 'n_len_100']].copy()
        #targets
        data2Y_test = data_next[['n_tweets']].copy()
        data3Y_test = data_next[['n_tweets']].copy()
        
        #create models
        # http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
        LR2model = LR()
        LR3model = LR()
        #http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
        LogR2model = LogR()
        LogR3model = LogR()
        # http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html
        SVR2model = SVR()
        SVR3model = SVR()
        # http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoLars.html
        # Note for whoever writes report: LARS stands for Least Angle Regression
        Lasso2model = lasso()
        Lasso3model = lasso()
        
        #fit models
        LR2model.fit(data2X_train, data2Y_train)
        LR3model.fit(data3X_train, data3Y_train)
        LogR2model.fit(data2X_train, data2Y_train.unstack())
        LogR3model.fit(data3X_train, data3Y_train.unstack())
        SVR2model.fit(data2X_train, data2Y_train.unstack())
        SVR3model.fit(data3X_train, data3Y_train.unstack())
        Lasso2model.fit(data2X_train, data2Y_train)
        Lasso3model.fit(data3X_train, data3Y_train)
        
        #score models using r^2 scoring, f/t value and p value
        # http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html
        
        #problem 2 Lin Reg
        LR2score.append(LR2model.score(data2X_test, data2Y_test))
        f, p = freg(data2X_test, data2Y_test.unstack())
        LR2fval = LR2fval + f
        LR2pval = LR2pval + p
        
        #problem 3 Lin Reg
        LR3score.append(LR3model.score(data3X_test, data3Y_test))
        f, p = freg(data3X_test, data3Y_test.unstack())
        LR3fval = LR3fval + f
        LR3pval = LR3pval + p
        
        #problem 2 Log Reg
        LogR2score.append(LogR2model.score(data2X_test, data2Y_test.unstack()))
        f, p = freg(data2X_test, data2Y_test.unstack())
        LogR2fval = LogR2fval + f
        LogR2pval = LogR2pval + p
        
        #problem 3 Log Reg
        LogR3score.append(LogR3model.score(data3X_test, data3Y_test.unstack()))
        f, p = freg(data3X_test, data3Y_test.unstack())
        LogR3fval = LogR3fval + f
        LogR3pval = LogR3pval + p
        
        #problem 2 SVR 
        SVR2score.append(SVR2model.score(data2X_test, data2Y_test.unstack()))
        f, p = freg(data2X_test, data2Y_test.unstack())
        SVR2fval = SVR2fval + f
        SVR2pval = SVR2pval + p
        
        #problem 3 SVR 
        SVR3score.append(SVR3model.score(data3X_test, data3Y_test.unstack()))
        f, p = freg(data3X_test, data3Y_test.unstack())
        SVR3fval = SVR3fval + f
        SVR3pval = SVR3pval + p
        
        #problem 2 lARS
        Lasso2score.append(Lasso2model.score(data2X_test, data2Y_test))
        f, p = freg(data2X_test, data2Y_test.unstack())
        Lasso2fval = Lasso2fval + f
        Lasso2pval = Lasso2pval + p
        
        #problem 3 lARS
        Lasso3score.append(Lasso3model.score(data3X_test, data3Y_test))
        f, p = freg(data3X_test, data3Y_test.unstack())
        predicted = Lasso3model.predict(data3X_test)
        Lasso3fval = Lasso3fval + f
        Lasso3pval = Lasso3pval + p
    
    # print results
    print("\tLinear Regression Mean Scores:")
    print("\t-Problem 2-")
    print("\t\tR^2 score: %5f" % np.mean(LR2score))
    print("\t\t" + "-"*30)
    for num, f in enumerate(LR2fval):
        print("\t\tT-test feature %d: %9f" % (num+1, f/len(LR2score)))
    print("\t\t" + "-"*30)
    for num, p in enumerate(LR2pval):
        print("\t\tP-value feature %d: %9f" % (num+1, p/len(LR2score)))
    print("\t-Problem 3-")
    print("\t\tR^2 score: %5f" % np.mean(LR3score))
    print("\t\t" + "-"*30)
    for num, f in enumerate(LR3fval):
        print("\t\tT-test feature %d: %9f" % (num+1, f/len(LR3score)))
    print("\t\t" + "-"*30)
    for num, p in enumerate(LR3pval):
        print("\t\tP-value feature %d: %9f" % (num+1, p/len(LR3score)))
    print('\n')
    
    print("\tLogistic Regression Mean Scores:")
    print("\t-Problem 2-")
    print("\t\tR^2 score: %5f" % np.mean(LogR2score))
    print("\t\t" + "-"*30)
    for num, f in enumerate(LogR2fval):
        print("\t\tT-test feature %d: %9f" % (num+1, f/len(LogR2score)))
    print("\t\t" + "-"*30)
    for num, p in enumerate(LogR2pval):
        print("\t\tP-value feature %d: %9f" % (num+1, p/len(LogR2score)))
    print("\t-Problem 3-")
    print("\t\tR^2 score: %5f" % np.mean(LogR3score))
    print("\t\t" + "-"*30)
    for num, f in enumerate(LogR3fval):
        print("\t\tT-test feature %d: %9f" % (num+1, f/len(LogR3score)))
    print("\t\t" + "-"*30)
    for num, p in enumerate(LogR3pval):
        print("\t\tP-value feature %d: %9f" % (num+1, p/len(LogR3score)))
    print('\n')
    
    print("\tSupport Vector Regression Mean Scores:")
    print("\t-Problem 2-")
    print("\t\tR^2 score: %5f" % np.mean(SVR2score))
    print("\t\t" + "-"*30)
    for num, f in enumerate(SVR2fval):
        print("\t\tT-test feature %d: %9f" % (num+1, f/len(SVR2score)))
    print("\t\t" + "-"*30)
    for num, p in enumerate(SVR2pval):
        print("\t\tP-value feature %d: %9f" % (num+1, p/len(SVR2score)))
    print("\t-Problem 3-")
    print("\t\tR^2 score: %5f" % np.mean(SVR3score))
    print("\t\t" + "-"*30)
    for num, f in enumerate(SVR3fval):
        print("\t\tT-test feature %d: %9f" % (num+1, f/len(SVR3score)))
    print("\t\t" + "-"*30)
    for num, p in enumerate(SVR3pval):
        print("\t\tP-value feature %d: %9f" % (num+1, p/len(SVR3score)))
    print('\n')
    
    print("\tLARS Lasso Mean Scores:")
    print("\t-Problem 2-")
    print("\t\tR^2 score: %5f" % np.mean(Lasso2score))
    print("\t\t" + "-"*30)
    for num, f in enumerate(Lasso2fval):
        print("\t\tT-test feature %d: %9f" % (num+1, f/len(Lasso2score)))
    print("\t\t" + "-"*30)
    for num, p in enumerate(Lasso2pval):
        print("\t\tP-value feature %d: %9f" % (num+1, p/len(Lasso2score)))
    print("\t-Problem 3-")
    print("\t\tR^2 score: %5f" % np.mean(Lasso3score))
    print("\t\t" + "-"*30)
    for num, f in enumerate(Lasso3fval):
        print("\t\tT-test feature %d: %9f" % (num+1, f/len(Lasso3score)))
    print("\t\t" + "-"*30)
    for num, p in enumerate(Lasso3pval):
        print("\t\tP-value feature %d: %9f" % (num+1, p/len(Lasso3score)))
    print('\n')
    
    # Graph predicted vs feature count
    plt.cla()
    #plt.yscale('log')
    plt.scatter(data3X_test[['sum_followers']], predicted)
    plt.title('Tweet Count vs sum_followers')
    plt.xlabel('sum_followers')
    plt.ylabel(c+'Number of Tweets')

    plt.savefig('graphs/count_vs_sum_'+c[5:-4])
    plt.show()
    
    plt.cla()
    #plt.yscale('log')
    plt.scatter(data3X_test[['sum_followers']], predicted)
    plt.title('Tweet Count vs n_users')
    plt.xlabel('sum_followers')
    plt.ylabel(c+'Number of Tweets')

    plt.savefig('graphs/count_vs_n_users_'+c[5:-4])
    plt.show()
    
    plt.cla()
    #plt.yscale('log')
    plt.scatter(data3X_test[['sum_followers']], predicted)
    plt.title('Tweet Count vs n_len_100')
    plt.xlabel('sum_followers')
    plt.ylabel(c+'Number of Tweets')

    plt.savefig('graphs/count_vs_n_len_100_'+c[5:-4])
    plt.show()
    
    #break #remove to run all for all files
    

data/tweets_#gohawks.csv

	Linear Regression Mean Scores:
	-Problem 2-
		R^2 score: -3.431273
		------------------------------
		T-test feature 1: 32.254497
		T-test feature 2: 12339.208428
		T-test feature 3:  2.607192
		T-test feature 4:       nan
		------------------------------
		P-value feature 1:  0.185462
		P-value feature 2:  0.000000
		P-value feature 3:  0.182427
		P-value feature 4:       nan
	-Problem 3-
		R^2 score: 0.830944
		------------------------------
		T-test feature 1: 32.254497
		T-test feature 2: 12339.208428
		T-test feature 3:  2.607192
		T-test feature 4:       nan
		T-test feature 5:  2.305868
		T-test feature 6: 26540.184413
		T-test feature 7: 645.459722
		T-test feature 8: 1392.503026
		------------------------------
		P-value feature 1:  0.185462
		P-value feature 2:  0.000000
		P-value feature 3:  0.182427
		P-value feature 4:       nan
		P-value feature 5:  0.451127
		P-value feature 6:  0.000000
		P-value feature 7:  0.000026
		P-value feature 8:  0.00

In [29]:
### problem 4
#1) 0-7
#2) 8-19
#3) 20-23
from os import listdir
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression as LR
from sklearn.linear_model import LassoLars as lasso
from sklearn.linear_model import LogisticRegression as LogR
from sklearn.feature_selection import f_regression as freg
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as m
from sklearn.cross_validation import cross_val_score as cv

# directory name
path = 'data/'

# hashtag name
hashtag = []

# csv file name
csv = []

files = listdir(path)

# append csv names into list
for f in files:
    if f.endswith(".csv"):
        hashtag.append(f[7:-4])
        csv.append(path + f)

# start the training per hashtag file

col_names = ["n_tweets", "n_retweets", "sum_followers", "max_followers", "hour", "avg_tweet_per_user", "n_users", "n_users_3", "n_len_100"]
for c in csv:
    print(c + "\n")
    csv_df = pd.read_csv(c, header=None, names=col_names)
    # aggregate by hour of day
    csv_df.sort_values(by='hour')
    period1 = csv_df[csv_df['hour'] < 8].copy()
    period2 = csv_df[csv_df['hour'] > 7].copy()
    period2 = period2[period2['hour']<20].copy()
    period3 = csv_df[csv_df['hour'] > 19].copy()
    
    period1_feat = period1[['n_retweets','sum_followers', 'max_followers', 'hour', 'avg_tweet_per_user', 'n_users', 'n_users_3', 'n_len_100']].copy()
    period1_tar = period1[['n_tweets']].copy()
    period2_feat = period2[['n_retweets','sum_followers', 'max_followers', 'hour', 'avg_tweet_per_user', 'n_users', 'n_users_3', 'n_len_100']].copy()
    period2_tar = period2[['n_tweets']].copy()
    period3_feat = period3[['n_retweets','sum_followers', 'max_followers', 'hour', 'avg_tweet_per_user', 'n_users', 'n_users_3', 'n_len_100']].copy()
    period3_tar = period3[['n_tweets']].copy()
    
    model = lasso()
    score1 = cv(model, period1_feat, period1_tar, scoring='mean_squared_error', cv=10, n_jobs=1)
    score2 = cv(model, period2_feat, period2_tar, scoring='mean_squared_error', cv=10, n_jobs=1)
    score3 = cv(model, period3_feat, period3_tar, scoring='mean_squared_error', cv=10, n_jobs=1)
    
    print("\tAverage prediction error for Period 1 %f:" % m.sqrt(abs(np.mean(score1))))
    print("\tAverage prediction error for Period 2 %f:" % m.sqrt(abs(np.mean(score2))))
    print("\tAverage prediction error for Period 3 %f:" % m.sqrt(abs(np.mean(score3))))
    print("-"*60)
    #break #remove to run all hashtags

data/tweets_#gohawks.csv

	Average prediction error for Period 1 27.448149:
	Average prediction error for Period 2 36.383078:
	Average prediction error for Period 3 16.337346:
------------------------------------------------------------
data/tweets_#gopatriots.csv

	Average prediction error for Period 1 61.608437:
	Average prediction error for Period 2 65.836415:
	Average prediction error for Period 3 10.785898:
------------------------------------------------------------
data/tweets_#nfl.csv

	Average prediction error for Period 1 51.328586:
	Average prediction error for Period 2 170.605770:
	Average prediction error for Period 3 27.154346:
------------------------------------------------------------
data/tweets_#patriots.csv

	Average prediction error for Period 1 84.086567:
	Average prediction error for Period 2 65.676033:
	Average prediction error for Period 3 16.494728:
------------------------------------------------------------
data/tweets_#sb49.csv

	Average prediction error fo

In [21]:
period2

,n_tweets,n_retweets,sum_followers,max_followers,hour,avg_tweet_per_user,n_users,n_users_3,n_len_100
0,1,2,749230320,749230320,14,1.000000,1,0,0
3,1,0,26556905,26556905,11,1.000000,1,0,1
4,1,0,2419495681,2419495681,12,1.000000,1,0,1
5,1,1,2419495681,2419495681,16,1.000000,1,0,1
6,1,1,266124599,266124599,10,1.000000,1,0,0
7,2,2,852924882,783479653,16,1.000000,2,0,2
8,1,1,570307870,570307870,8,1.000000,1,0,0
9,1,2,1040232535,1040232535,11,1.000000,1,0,1
10,1,0,2419495681,2419495681,17,1.000000,1,0,1
11,1,1,2374402878,2374402878,10,1.000000,1,0,1
